In [1]:
%matplotlib inline

In [2]:
import lsst.sims.skybrightness as sb
import lsst.sims.photUtils.Bandpass as Bandpass
import os
import numpy as np
import matplotlib.pylab as plt
import healpy as hp

In [3]:
sm = sb.SkyModel(mags=True, twilight=False, zodiacal=False, moon=True)
nside = 16
hpmap = np.zeros(hp.nside2npix(nside))
lat, ra = hp.pix2ang(nside, np.arange(hpmap.size))
dec = np.pi/2-lat
good = np.where(dec < 0)
dec = dec[good]
ra = ra[good]
print 'computing at %i points on the sky' % ra.size

computing at 1504 points on the sky


In [4]:
# Let's pull a large number of dark time mjds to test
data,ack = sb.allSkyDB(100, sqlQ='select mjd from dates where sunAlt < %f and moonAlt < %f' % (np.radians(-20.), 0.), 
                   dtypes=zip(['mjd'],[float]))

In [5]:
data['mjd']

array([ 56948.019861,  56948.020683,  56948.021493, ...,  57182.17353 ,
        57182.174352,  57182.175162])

In [6]:
import timeit
def runStuff(sm, mjd, n=10):
    for i in np.arange(n):
        sm.setRaDecMjd(ra,dec, mjd[i])

npts = [10,100,1000]
timeResults = []
for npt in npts:
    time = timeit.timeit("runStuff(sm,data['mjd'], n=%i)" % npt, number=1, setup="from __main__ import runStuff,sm, data")
    timeResults.append(time)
    


In [7]:
timeResults

[0.033425092697143555, 0.31086111068725586, 3.137404203414917]

In [8]:
print 'Compute time for 2.5 million 6-filter, half-sky, moon down calcs = %f hours' % (timeResults[-1]/1e3*2.5e6/3600.)

Compute time for 2.5 million 6-filter, half-sky, moon down calcs = 2.178753 hours


In [9]:
# Now with the moon up
data,ack = sb.allSkyDB(100, sqlQ='select mjd from dates where sunAlt < %f and moonAlt > %f' % (np.radians(-20.), np.radians(10.)), 
                   dtypes=zip(['mjd'],[float]))

In [10]:
timeResults = []
for npt in npts:
    time = timeit.timeit("runStuff(sm,data['mjd'], n=%i)" % npt, number=1, setup="from __main__ import runStuff,sm, data")
    timeResults.append(time)
timeResults

/Users/yoachim/lsst/DarwinX86/healpy/1.8.1+14/lib/python/healpy-1.8.1-py2.7-macosx-10.5-x86_64.egg/healpy/pixelfunc.py:1045: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if phi == None:


[0.05101490020751953, 0.48423194885253906, 4.847275972366333]

In [11]:
print 'Compute time for 2.5 million 6-filter, half-sky, moon-up calcs = %f hours' % (timeResults[-1]/1e3*2.5e6/3600.)

Compute time for 2.5 million 6-filter, half-sky, moon-up calcs = 3.366164 hours
